In [3]:
from google.colab import drive
import os, pathlib
from pathlib import Path
import pandas as pd
import numpy as np

In [1]:
drive.mount('/content/drive')

REPO_URL   = "https://github.com/Svanzi/Tennis-Shot-Recognition.git"
BASE       = "/content/drive/MyDrive/University/Deep"
SPARSE_DIR = "Tennis-Shot-Recognition"   # repo "leggera" solo per i dati
TARGET     = f"{BASE}/{SPARSE_DIR}"

os.makedirs(BASE, exist_ok=True)

# Clona in modalità sparse solo la prima volta
if not os.path.exists(TARGET):
    %cd {BASE}
    !git clone --filter=blob:none --sparse {REPO_URL} {SPARSE_DIR}
    %cd {SPARSE_DIR}
    !git sparse-checkout set DataSet
else:
    %cd {TARGET}
    !git pull

# Path da usare nel notebook
from pathlib import Path
DATA_DIR = Path(TARGET) / "DataSet"
print("DATA_DIR =", DATA_DIR)

Mounted at /content/drive
/content/drive/MyDrive/University/Deep/Tennis-Shot-Recognition_ds
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 2 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (2/2), 266 bytes | 19.00 KiB/s, done.
Resolving deltas: 100% (1/1), completed with 1 local object.
From https://github.com/Svanzi/Tennis-Shot-Recognition
   07eb185..9dcb9e3  main       -> origin/main
Updating 07eb185..9dcb9e3
Fast-forward
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 1 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (1/1), 1.45 KiB | 36.00 KiB/s, done.
 Shot_Classification.ipynb | 161 ++++++++++++++++++++++++++++++++++------------
 1 file changed, 120 insertions(+), 41 deletions(-)
DATA_DIR = /content/drive/MyDrive/University/Deep/Tennis-Shot-Recognition_ds/DataSet


In [ ]:
DATA_DIR = Path("/content/drive/MyDrive/University/Deep/Tennis-Shot-Recognition/DataSet")


files = sorted(DATA_DIR.glob("**/*.csv"))
print(len(files), "file")

# carica ogni csv in un np.array (righe = tempo, colonne = feature)
seqs = [pd.read_csv(f).to_numpy() for f in files]  # lista di array (lunghezze possono variare)
